## Habit Tracker 

Advanced Topics Used:
1. File I/O (Week 9) - JSON and text file operations
2. Multi-Dimensional Lists (Week 12) - 2D matrix analysis for habit patterns

In [2]:
import json
import os
from datetime import datetime, timedelta

In [3]:
def init_files():
    """Initialize required files if they don't exist"""
    if not os.path.exists("habits.json"):
        with open("habits.json", "w") as f:
            json.dump({}, f)
    
    if not os.path.exists("records.json"):
        with open("records.json", "w") as f:
            json.dump({}, f)
    
    if not os.path.exists("achievements.txt"):
        with open("achievements.txt", "w") as f:
            f.write("=== YOUR ACHIEVEMENTS ===\n")

## FILE I/O FUNCTIONS 

In [4]:
def load_data(filename):
    """Load data from JSON file"""
    with open(filename, "r") as f:
        return json.load(f)

def save_data(filename, data):
    """Save data to JSON file"""
    with open(filename, "w") as f:
        json.dump(data, f, indent=2)

def log_achievement(message):
    """Append achievement to text file"""
    with open("achievements.txt", "a") as f:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M")
        f.write(f"[{timestamp}] {message}\n")

## MULTI-DIMENSIONAL LIST ANALYSIS

In [5]:
def analyze_patterns():
    """
    Analyze habit patterns using 2D lists
    Matrix structure: rows = dates, columns = habits
    """
    records = load_data("records.json")
    habits = load_data("habits.json")
    
    if not records:
        print("\n📊 No data available for analysis yet!")
        input("\nPress Enter to continue...")
        return
    
    # Build 2D matrix: rows=dates, columns=habits
    dates = sorted(records.keys())
    habit_names = list(habits.keys())
    
    # Create 2D list (matrix)
    matrix = []
    for date in dates:
        row = []
        for habit in habit_names:
            if habit in records[date]:
                row.append(1)  # Completed
            else:
                row.append(0)  # Not completed
        matrix.append(row)
    
    print("\n" + "="*60)
    print("📊 HABIT PATTERN ANALYSIS (Multi-Dimensional Lists)")
    print("="*60)
    
    # Analysis 1: Completion count per habit
    print("\n1️⃣  Total completions per habit:")
    for i, habit in enumerate(habit_names):
        count = sum(row[i] for row in matrix)
        percentage = (count / len(matrix)) * 100 if matrix else 0
        print(f"   • {habit}: {count} times ({percentage:.1f}%)")
    
    # Analysis 2: Best days (days with most habits completed)
    print("\n2️⃣  Most productive days:")
    day_scores = []
    for i, row in enumerate(matrix):
        total = sum(row)
        day_scores.append((dates[i], total))
    
    day_scores.sort(key=lambda x: x[1], reverse=True)
    for date, count in day_scores[:5]:
        print(f"   • {date}: {count} habits completed")
    
    # Analysis 3: Consistency score (how evenly distributed)
    print("\n3️⃣  Habit consistency scores:")
    for i, habit in enumerate(habit_names):
        completions = [row[i] for row in matrix]
        consistency = (sum(completions) / len(completions)) * 100
        print(f"   • {habit}: {consistency:.1f}% consistent")
    
    input("\nPress Enter to continue...")

## CORE FEATURES

In [6]:
def show_menu():
    """Display main menu"""
    print("\n" + "="*60)
    print("🎯 HABIT TRACKER - Main Menu")
    print("="*60)
    print("1. View all habits")
    print("2. Add new habit")
    print("3. Log today's habit")
    print("4. Analyze habit patterns ⭐ (Multi-Dimensional Analysis)")
    print("5. View streaks")
    print("6. View achievements")
    print("7. Exit")
    print("="*60)

In [7]:
def add_habit():
    """Add a new habit with resistance and confidence levels"""
    habits = load_data("habits.json")
    
    print("\n" + "="*60)
    print("➕ ADD NEW HABIT")
    print("="*60)
    
    name = input("Habit name: ").strip()
    
    if not name:
        print("❌ Habit name cannot be empty!")
        input("\nPress Enter to continue...")
        return
    
    if name in habits:
        print(f"❌ Habit '{name}' already exists!")
        input("\nPress Enter to continue...")
        return
    
    try:
        resistance = int(input("Resistance level (1-10): "))
        confidence = int(input("Confidence level (1-10): "))
        
        if not (1 <= resistance <= 10 and 1 <= confidence <= 10):
            print("❌ Values must be between 1 and 10!")
            input("\nPress Enter to continue...")
            return
        
        habits[name] = {
            "created": datetime.now().strftime("%Y-%m-%d"),
            "resistance": resistance,
            "confidence": confidence
        }
        
        save_data("habits.json", habits)
        print(f"\n✅ Successfully added habit: {name}")
        
    except ValueError:
        print("❌ Please enter valid numbers!")
    
    input("\nPress Enter to continue...")

In [8]:
def log_habit():
    """Log habit completion for today"""
    habits = load_data("habits.json")
    records = load_data("records.json")
    
    if not habits:
        print("\n❌ No habits found! Please add a habit first.")
        input("\nPress Enter to continue...")
        return
    
    print("\n" + "="*60)
    print("📝 LOG TODAY'S HABIT")
    print("="*60)
    
    print("\nAvailable habits:")
    habit_list = list(habits.keys())
    for i, h in enumerate(habit_list, 1):
        print(f"  {i}. {h}")
    
    try:
        choice = int(input("\nSelect habit number: ")) - 1
        
        if choice < 0 or choice >= len(habit_list):
            print("❌ Invalid selection!")
            input("\nPress Enter to continue...")
            return
        
        habit_name = habit_list[choice]
        mood = input("How did you feel? (happy/calm/anxious/focused/tired): ").strip().lower()
        
        valid_moods = ["happy", "calm", "anxious", "focused", "tired"]
        if mood not in valid_moods:
            mood = "neutral"
        
        today = datetime.now().strftime("%Y-%m-%d")
        if today not in records:
            records[today] = {}
        
        records[today][habit_name] = mood
        save_data("records.json", records)
        
        print(f"\n✅ Logged: {habit_name} (mood: {mood})")
        
        # Calculate streak
        streak = calculate_streak(habit_name, records)
        print(f"🔥 Current streak: {streak} days!")
        
        # Check for achievements
        if streak == 7:
            achievement = f"🏆 7-day streak achieved for {habit_name}!"
            log_achievement(achievement)
            print(f"\n{achievement}")
        elif streak == 30:
            achievement = f"🏆 30-day streak achieved for {habit_name}!"
            log_achievement(achievement)
            print(f"\n{achievement}")
        
    except (ValueError, IndexError):
        print("❌ Invalid input!")
    
    input("\nPress Enter to continue...")

In [9]:
def calculate_streak(habit_name, records):
    """Calculate current consecutive days for a habit"""
    streak = 0
    current = datetime.now().date()
    
    for _ in range(365):  # Check up to one year
        date_str = current.strftime("%Y-%m-%d")
        if date_str in records and habit_name in records[date_str]:
            streak += 1
            current -= timedelta(days=1)
        else:
            break
    
    return streak

In [10]:
def view_habits():
    """Display all habits with their details"""
    habits = load_data("habits.json")
    
    if not habits:
        print("\n📝 No habits yet. Add your first habit!")
    else:
        print("\n" + "="*60)
        print("📝 YOUR HABITS")
        print("="*60)
        for name, info in habits.items():
            print(f"\n🔹 {name}")
            print(f"   Created: {info['created']}")
            print(f"   Resistance: {info['resistance']}/10")
            print(f"   Confidence: {info['confidence']}/10")
    
    input("\nPress Enter to continue...")

In [11]:
def view_streaks():
    """Display current streaks for all habits"""
    habits = load_data("habits.json")
    records = load_data("records.json")
    
    if not habits:
        print("\n🔥 No habits to track yet!")
    else:
        print("\n" + "="*60)
        print("🔥 CURRENT STREAKS")
        print("="*60)
        for habit_name in habits.keys():
            streak = calculate_streak(habit_name, records)
            emoji = "🔥" if streak >= 7 else "⭐" if streak >= 3 else "💪"
            print(f"{emoji} {habit_name}: {streak} days")
    
    input("\nPress Enter to continue...")

In [12]:
def view_achievements():
    """Display all unlocked achievements"""
    if os.path.exists("achievements.txt"):
        print("\n" + "="*60)
        print("🏆 YOUR ACHIEVEMENTS")
        print("="*60)
        with open("achievements.txt", "r") as f:
            content = f.read()
            if len(content.strip()) <= len("=== YOUR ACHIEVEMENTS ==="):
                print("\n🏆 No achievements yet! Keep building your habits!")
            else:
                print(content)
    else:
        print("\n🏆 No achievements yet!")
    
    input("\nPress Enter to continue...")

In [ ]:
## DEMO DATA PREPARATION (Run this before presentation)

def prepare_demo_data():
    """
    Generate sample data for demonstration
    Creates 3 weeks of habit tracking data
    """
    import random
    
    # Initialize files
    init_files()
    
    # Clean existing data and create fresh demo data
    habits = {
        "Morning run": {
            "created": "2025-10-13",
            "resistance": 4,
            "confidence": 7
        },
        "Reading 20 min": {
            "created": "2025-10-03",
            "resistance": 3,
            "confidence": 8
        },
        "Meditation 10 min": {
            "created": "2025-10-18",
            "resistance": 5,
            "confidence": 6
        }
    }
    
    # Generate records for past 3 weeks
    records = {}
    habit_names = list(habits.keys())
    moods = ["happy", "calm", "focused", "relaxed", "tired"]
    
    # Start from 21 days ago
    start_date = datetime.now() - timedelta(days=21)
    
    for day in range(21):
        current_date = start_date + timedelta(days=day)
        date_str = current_date.strftime("%Y-%m-%d")
        records[date_str] = {}
        
        # Each day, complete 1-2 habits (not all, to show variation)
        habits_to_complete = random.sample(habit_names, random.randint(1, 2))
        
        for habit in habits_to_complete:
            # Make Morning run more frequent (70% chance)
            if habit == "Morning run":
                if random.random() < 0.7:
                    records[date_str][habit] = random.choice(moods)
            # Reading is consistent (80% chance)
            elif habit == "Reading 20 min":
                if random.random() < 0.8:
                    records[date_str][habit] = random.choice(moods)
            # Meditation is less frequent (50% chance)
            else:
                if random.random() < 0.5:
                    records[date_str][habit] = random.choice(moods)
    
    # Add some achievements
    achievements_content = """=== YOUR ACHIEVEMENTS ===
[2025-10-20 08:15] 🏆 7-day streak achieved for Morning run!
[2025-10-15 19:30] 🏆 7-day streak achieved for Reading 20 min!
"""
    
    # Save all data
    save_data("habits.json", habits)
    save_data("records.json", records)
    
    with open("achievements.txt", "w") as f:
        f.write(achievements_content)
    
    print("✅ Demo data prepared successfully!")
    print(f"   - {len(habits)} habits created")
    print(f"   - {len(records)} days of records")
    print("   - Ready for presentation!")
    print("\n💡 Tip: Now you can run main() to start the demo")

# Uncomment the line below to generate demo data:
# prepare_demo_data()


## MAIN PROGRAM

In [13]:
def main():
    """Main program loop"""
    init_files()
    
    print("\n" + "="*60)
    print("🎯 WELCOME TO HABIT TRACKER")
    print("="*60)
    print("Build better habits, track your progress, unlock achievements!")
    input("\nPress Enter to start...")
    
    while True:
        show_menu()
        choice = input("\nEnter your choice (1-7): ").strip()
        
        if choice == "1":
            view_habits()
        elif choice == "2":
            add_habit()
        elif choice == "3":
            log_habit()
        elif choice == "4":
            analyze_patterns()  # Multi-Dimensional List analysis
        elif choice == "5":
            view_streaks()
        elif choice == "6":
            view_achievements()
        elif choice == "7":
            print("\n👋 Thank you for using Habit Tracker!")
            print("Keep up the great work! 💪")
            break
        else:
            print("\n❌ Invalid choice! Please enter 1-7.")

In [ ]:
if __name__ == "__main__":
    main()


🎯 WELCOME TO HABIT TRACKER
Build better habits, track your progress, unlock achievements!

🎯 HABIT TRACKER - Main Menu
1. View all habits
2. Add new habit
3. Log today's habit
4. Analyze habit patterns ⭐ (Multi-Dimensional Analysis)
5. View streaks
6. View achievements
7. Exit

❌ Invalid choice! Please enter 1-7.

🎯 HABIT TRACKER - Main Menu
1. View all habits
2. Add new habit
3. Log today's habit
4. Analyze habit patterns ⭐ (Multi-Dimensional Analysis)
5. View streaks
6. View achievements
7. Exit

📝 YOUR HABITS

🔹 Morning run
   Created: 2025-10-13
   Resistance: 4/10
   Confidence: 7/10

🔹 Reading 20 min
   Created: 2025-10-03
   Resistance: 3/10
   Confidence: 8/10

🔹 Meditation 10 min
   Created: 2025-10-18
   Resistance: 5/10
   Confidence: 6/10

🎯 HABIT TRACKER - Main Menu
1. View all habits
2. Add new habit
3. Log today's habit
4. Analyze habit patterns ⭐ (Multi-Dimensional Analysis)
5. View streaks
6. View achievements
7. Exit

➕ ADD NEW HABIT

✅ Successfully added habit: 3

🎯